## Installation

In [ ]:
%pip install -q pyomo
!apt-get install coinor-cbc

#### Initializing libraries and model

In [ ]:
from pyomo.environ import *
import math
model=ConcreteModel()
model.constraints=ConstraintList()

Add initial variables here, like importing the game files, the rules, etc, which are constants,or decided by the game

In [ ]:
n=1000                              #Number of match ticks
E=[i for i in range (n)]            #Enemy strength for each attack tick
T=500                               #Treasure required to reach victory
V=[i for i in range(n)]             #Number of jobs in each match tick
Cg=100                              #Amount of gold one can generate in each job for one job tick
Cf=100                              #Amount of food one can generate in each job for one job tick
Ct=100                              #Amount of treasure one can generate in each job for one job tick
Tg=20                               #Amount of gold required to train one soldier
Tf=17                               #Amount of food required to train one soldier
M=1000000                           #The big M we will be using
Wininc=100000                       #The army winning increase, needs to be optimized for better solution
Str_Sol=72                          #Strength of one soldier

#### Instruction for modelling
Add variables by ``model.x=Var(range(_dimension_),domain=[_the domain of the variable_])``

In [ ]:
model.food_in=Var(range(n), domain=NonNegativeIntegers)                 #Number of jobs allocated to food in each tick
model.gold_in=Var(range(n),domain=NonNegativeIntegers)                  #Number of jobs allocated to gold in each tick
model.treasure_in=Var(range(n),domain=NonNegativeIntegers)              #Number of jobs allocated to treasure in each tick
model.total_gold=Var(range(n),domain=NonNegativeIntegers)               #Represents gold before ith attack {deducting all costs}
model.total_food=Var(range(n),domain=NonNegativeIntegers)               #Represents food before ith attack {deducting all costs}
model.total_treasure=Var(range(n),domain=NonNegativeIntegers)           #Represents treasure before ith attack {deducting all costs}
model.train=Var(range(n),domain=NonNegativeIntegers)                    #Represents the number of soldiers trained before ith attack
model.army=Var(range(n),domain=NonNegativeIntegers)                     #Represents the strength of total army before ith attack
model.armyV=Var(range(n),domain=NonNegativeIntegers)                    #Represents the strength of total army after ith attack
model.W=Var(range(n),domain=Binary)                                     #Represents binary which denotes whether won at or before ith attack or not
model.r=Var(range(n),domain=[1.0000,0.8000,0.6667,0.5714,0.5000,0.3333,0.2000,0]) #Represents the  fraction of army which should survive

Objective function

In [ ]:
model.obj = Objective(expr = n-sum(model.W), sense = minimize)          #Gives the fastest possible win, collecting T amounts of treasure

Add constraints by ``model.constraint.add(expr=...)``

In [ ]:
for i in range(1,n):
    model.constraints.add(expr=model.total_gold[i]==model.total_gold[i-1]+Cg*model.gold_in-Tg*model.train[i])
    model.constraints.add(expr=model.total_food[i]==model.total_food[i-1]+Cf*model.food_in-Tf*model.train[i])
    model.constraints.add(expr=model.total_treasure[i]==model.total_treasure[i-1]+Ct*model.treasure_in[i])
    model.constraints.add(expr=model.army[i]==model.armyV[i-1]+Str_Sol*model.train[i]+Wininc*model.W[i])
    model.constraints.add(expr=model.armyV[i]>=0)
    model.constraints.add(expr=model.army[i]>=model.E[i])

model.constraints.add(expr=model.total_treasure[0]==0)

for i in range(n):
    model.constraints.add(expr=model.total_gold[i]>=0)
    model.constraints.add(expr=model.total_food[i]>=0)
    model.constraints.add(expr=model.food_in[i]+model.gold_in[i]+model.treasure_in[i]==V[i])
    model.constraints.add(expr=model.total_treasure[i]<=M*model.W[i]+T-1)
    model.constraints.add(expr=model.total_treasure[i]>=-M*(1-model.W[i])+T-1)

## Solver using CBC

In [ ]:
opt = SolverFactory('cbc')
result=opt.solve(model,tee=True)
print('Solver status: ',result.solver.status)
print('Solver termination condition: ',result.solver.termination_condition)

In [ ]:
print('Optimal N',model.obj())